References : https://github.com/ukairia777/finance_sentiment_corpus

# 한국어 금융 뉴스 긍정, 부정 분류

In [ ]:
!pip install transformers
!pip install datasets

## 데이터셋 로드 및 구조 확인

In [ ]:
!wget https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv

--2024-07-30 14:26:03--  https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1319001 (1.3M) [text/plain]
Saving to: ‘finance_data.csv’

finance_data.csv    100%[===================>]   1.26M  --.-KB/s    in 0.009s  

2024-07-30 14:26:04 (142 MB/s) - ‘finance_data.csv’ saved [1319001/1319001]



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('finance_data.csv')

In [ ]:
print('샘플의 개수 :', len(df))

샘플의 개수 : 4846


In [ ]:
df.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [ ]:
df['labels'].value_counts()

labels
neutral     2879
positive    1363
negative     604
Name: count, dtype: int64

In [ ]:
df['labels'] = df['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
df.head()

,labels,sentence,kor_sentence
0,0,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [ ]:
df.to_csv('finance_data.csv', index=False, encoding='utf-8-sig')

In [ ]:
from datasets import load_dataset

In [ ]:
all_data = load_dataset(
        "csv",
        data_files={
            "train": "finance_data.csv",
        },
    )

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
all_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'sentence', 'kor_sentence'],
        num_rows: 4846
    })
})

In [ ]:
cs = all_data['train'].train_test_split(0.2, seed=777)
train_cs = cs["train"]
test_cs = cs["test"]

In [ ]:
train_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3876
})

In [ ]:
test_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})

In [ ]:
# 훈련 데이터를 다시 8:2로 분리 후 훈련 데이터와 검증 데이터로 저장
cs = train_cs.train_test_split(0.2, seed=777)
train_cs = cs["train"]
valid_cs = cs["test"]

In [ ]:
# 훈련 데이터
train_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3100
})

In [ ]:
# 검증 데이터
valid_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 776
})

In [ ]:
# 테스트 데이터
test_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})

In [ ]:
print('두번째 샘플 출력 :', train_cs['kor_sentence'][1])
print('두번째 샘플의 레이블 출력 :', train_cs['labels'][1])

두번째 샘플 출력 : 알마 미디어 코퍼레이션 사업 ID 1944757-4의 주식 자본금은 44,767,513.80유로이며 74,612,523주로 나뉜다.
두번째 샘플의 레이블 출력 : 0


## 데이터셋 전처리

In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import torch

# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

In [ ]:
# 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 `[CLS] 문장 [SEP]` 구조를 만듭니다.

train_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', train_cs['kor_sentence']))
validation_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', valid_cs['kor_sentence']))
test_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', test_cs['kor_sentence']))

In [ ]:
train_labels = train_cs['labels']
validation_labels = valid_cs['labels']
test_labels = test_cs['labels']

In [ ]:
train_sentences[10:15]

['[CLS] 핀란드의 라우트 프리시젼은 멕시코와 미국으로부터 대규모 유리 배치 공장 및 모르타르 공장을 수주했다. [SEP]',
 '[CLS] `` 우리는 K-city market 고객들에게 흥미롭고 주제적인 선택을 제공함으로써 그들에게 서비스를 제공하고 싶습니다. [SEP]',
 '[CLS] 2007년에도 후타마키는 유기성장을 위한 투자를 계속할 예정이다. [SEP]',
 '[CLS] 건물 및 주택 개량 무역에서 순매출은 총 1,173만 유로로 전년도의 1,566만 유로에 비해 감소했다. [SEP]',
 '[CLS] 우리는 시스템의 구현 모델을 개발하기 위해 장기적인 투자를 해왔습니다. [SEP]']

In [ ]:
train_labels[10:15]

[1, 0, 0, 2, 0]

## BERT 토크나이저를 이용한 전처리

In [ ]:
# 한국어 BERT 중 하나인 'klue/bert-base'를 사용.
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
sent = tokenizer.tokenize('[CLS] 우리는 시스템의 구현 모델을 개발하기 위해 장기적인 투자를 해왔습니다. [SEP]')
print(sent)

['[CLS]', '우리', '##는', '시스템', '##의', '구현', '모델', '##을', '개발', '##하기', '위해', '장기', '##적인', '투자', '##를', '해왔', '##습', '##니다', '.', '[SEP]']


In [ ]:
encoding_sent = tokenizer.convert_tokens_to_ids(sent)
print(encoding_sent)

[2, 3616, 2259, 4119, 2079, 6948, 4347, 2069, 3720, 31302, 3627, 4376, 31221, 3703, 2138, 19540, 2219, 3606, 18, 3]


In [ ]:
max_len = 128

def data_to_tensor (sentences, labels, max_len):
  # 정수 인코딩 과정. 각 텍스트를 토큰화한 후에 Vocabulary에 맵핑되는 정수 시퀀스로 변환한다.
  # ex) ['안녕하세요'] ==> ['안', '녕', '하세요'] ==> [231, 52, 45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences는 패딩을 위한 모듈. 주어진 최대 길이를 위해서 뒤에서 0으로 채워준다.
  # ex) [231, 52, 45] ==> [231, 52, 45, 0, 0, 0]
  input_ids = pad_sequences(input_ids, maxlen=max_len, dtype="long", truncating="post", padding="post")

  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i > 0) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks

In [ ]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels, max_len)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels, max_len)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels, max_len)

In [ ]:
len(train_inputs)

3100

In [ ]:
tokenizer.decode([2])

'[CLS]'

In [ ]:
tokenizer.decode([3])

'[SEP]'

In [ ]:
print('정수 인코딩 결과:', test_inputs[0])
print('-' * 20)
print('원본 문장 복원 결과:', tokenizer.decode(test_inputs[0]))
print('-' * 20)
print('어텐션 마스크:', test_masks[0])
print('-' * 20)
print('샘플의 길이:', len(test_inputs[0]))
print('-' * 20)
print('레이블:', test_labels[0])

정수 인코딩 결과: tensor([    2,  4400,  3633,    18,  6264,  2067,     1,    22,    18,  6946,
         2006, 11453,  4720,  2470,  4056,    18,  7725,  2298,  2200,    16,
           51,  2110,  2312,  1915,  2942,  2089,  3912, 18618,  2259,    20,
           18,  5547,  2006, 11453,  4131,  2470,    22,    16,  7569,  2049,
           18,  3909,    16,    51,  2110,  2312,  1915,  2942,  2089,  2259,
           20,    18,  6350,  2006, 11453,  4720,  2470,    29,    16, 23010,
           18,  7948,  2298,  2200,  2170,  4083,  2367,  2062,    18,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,    

In [ ]:
tokenizer.decode([2])

'[CLS]'

In [ ]:
tokenizer.decode([3])

'[SEP]'

## 데이터의 배치화

In [ ]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
3100/32

96.875

In [ ]:
len(train_dataloader)

97

In [ ]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
print('훈련 데이터의 크기:', len(train_labels))
print('검증 데이터의 크기:', len(validation_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 3100
검증 데이터의 크기: 776
테스트 데이터의 크기: 970


In [ ]:
len(train_dataloader)

97

In [ ]:
3100 / 32

96.875

## GPU가 정상 셋팅되었는지 확인.  
Colab에서 GPU를 사용하기 위해서는 아래와 같이 설정이 되어있어야만 합니다.  

* 런타임 > 런타임 유형 변경 > 하드웨어 가속기 > 'GPU' 선택

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


## 모델 로드하기

In [ ]:
num_labels = 3

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels)
model.cuda()

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## 모델 학습

참고: https://wikidocs.net/21670

In [ ]:
# 옵티마이저 선택
optimizer = AdamW(model.parameters(), lr = 2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 몇 번의 에포크(전체 데이터에 대한 학습 횟수)를 할 것인지 선택
epochs = 3

In [ ]:
def metrics(predictions, labels):
    """
    분류 모델의 성능을 평가하는 메트릭을 계산하는 함수.

    Parameters:
    - predictions: 모델에 의해 예측된 레이블의 배열. 모든 예측값은 실제 레이블과 같은 형태를 가진다.
    - labels: 실제 레이블의 배열. 각 요소는 정답 클래스를 나타낸다.

    Returns:
    - metrics: 다음을 포함하는 딕셔너리
        - 'accuracy': 모델의 정확도. 모든 예측 중 정확한 예측의 비율.
        - 'f1_macro': F1 점수의 매크로 평균. 각 클래스를 동등하게 중요시하여 계산된 평균.
        - 'f1_micro': F1 점수의 마이크로 평균. 전체적인 정답과 오답의 비율로 계산된 평균, 이는 불균형 클래스에서 유용하다.
        - 'f1_weighted': F1 점수의 가중치 평균. 각 클래스에 속하는 샘플의 수에 따라 가중치를 적용하여 계산된 평균.

    메트릭 설명:
    - 정확도(accuracy): 모든 예측 중 정확하게 예측된 비율. 전체적인 모델의 정확성을 나타낸다.
    - F1 점수: 정밀도(precision)와 재현율(recall)의 조화 평균. 클래스 간의 균형을 고려한 성능 지표.
      - 매크로 평균(macro average): 각 클래스에 대한 F1 점수를 동일하게 취급하고, 이들의 평균을 계산.
      - 마이크로 평균(micro average): 전체적인 TP(True Positive), FP(False Positive), FN(False Negative)을 고려하여 F1 점수를 계산.
      - 가중치 평균(weighted average): 각 클래스의 샘플 수를 가중치로 사용하여 F1 점수를 계산.
    """
    y_pred = predictions
    y_true = labels

    # 사용 가능한 메트릭들을 사용한다.
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)

    # 메트릭 결과에 대해서 리턴
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average}

    return metrics

In [ ]:
def train_epoch(model, train_dataloader, optimizer, device):
    """
    하나의 에포크 동안 모델을 학습시키는 함수입니다.

    Parameters:
    model (torch.nn.Module): 학습시킬 모델 객체.
    train_dataloader (torch.utils.data.DataLoader): 학습 데이터셋의 DataLoader.
    optimizer (torch.optim.Optimizer): 최적화 알고리즘을 구현하는 객체.
    device (torch.device): 학습에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 학습 손실값.
    """

    total_train_loss = 0  # 학습 손실을 누적할 변수 초기화
    model.train()  # 모델을 학습 모드로 설정

    # 학습 데이터로더를 순회하며 배치 단위로 학습
    for step, batch in tqdm(enumerate(train_dataloader), desc="Training Batch"):
        batch = tuple(t.to(device) for t in batch)  # DataLoader에서 배치를 받아 각 텐서를 지정된 장치로 이동
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 입력 ID, 마스크, 라벨 추출

        # 모델에 배치를 전달하여 손실값 계산
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # 손실값 추출
        loss = outputs.loss

        optimizer.zero_grad()  # 그래디언트 초기화. // 미분값을 0으로 초기회
        loss.backward()  # 역전파를 통해 그래디언트 계산 // 미분
        optimizer.step()  # 매개변수 업데이트 // 미분 값을 바탕으로 W, b를 업데이트

        total_train_loss += loss.item()  # 총 손실에 더함

    avg_train_loss = total_train_loss / len(train_dataloader)  # 평균 학습 손실 계산

    return avg_train_loss  # 평균 학습 손실 반환

In [ ]:
def evaluate(model, validation_dataloader, device):
    """
    모델을 사용하여 검증 데이터셋에 대한 평가를 수행하는 함수입니다.

    Parameters:
    model (torch.nn.Module): 평가할 모델 객체.
    validation_dataloader (torch.utils.data.DataLoader): 검증 데이터셋의 DataLoader.
    device (torch.device): 평가에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 검증 손실값.
    dict: 다양한 평가 지표(metrics)에 대한 값들을 담은 사전.
    """

    model.eval()  # 모델을 평가 모드로 설정

    total_eval_loss = 0  # 검증 손실을 누적할 변수 초기화
    predictions, true_labels = [], []  # 예측값과 실제 라벨값을 저장할 리스트 초기화

    # 검증 데이터로더를 순회하며 배치 단위로 평가
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)  # 배치 데이터를 디바이스로 이동
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 입력 ID, 마스크, 라벨 추출

        with torch.no_grad():  # 그래디언트 계산을 수행하지 않음
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # 모델 출력에서 손실값 추출 (수정된 부분)
        if outputs.loss is not None:
            loss = outputs.loss
            total_eval_loss += loss.item()  # 총 손실에 더함

        logits = outputs.logits.detach().cpu().numpy()  # 모델 예측값(로짓)을 numpy 배열로 변환
        label_ids = b_labels.to('cpu').numpy()  # 실제 라벨값을 numpy 배열로 변환

        # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정 (예시: logits = [3.513, -0.309, -2.111] ==> 예측: 0)
        predictions.extend(np.argmax(logits, axis=1).flatten()) # 예측된 클래스를 리스트에 추가
        true_labels.extend(label_ids.flatten()) # 실제 레이블 값을 리스트에 추가

    eval_metrics = metrics(predictions, true_labels)

    return total_eval_loss / len(validation_dataloader), eval_metrics

In [ ]:
# 최소 검증 손실 초기화
min_val_loss = float('inf')

# 메인 학습 & 평가 루프
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))

    # 학습 단계
    train_epoch(model, train_dataloader, optimizer, device)

    print("\nRunning Validation...")
    # 검증 단계
    avg_val_loss, eval_metrics = evaluate(model, validation_dataloader, device)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
    print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
    print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
    print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))

    # 검증 손실이 현재까지의 최소값보다 작은 경우 체크포인트 저장
    if avg_val_loss < min_val_loss:
        print(f"Validation loss decreased ({min_val_loss:.2f} --> {avg_val_loss:.2f}).  Saving model ...")
        # 베스트 모델 저장
        torch.save(model.state_dict(), 'model_checkpoint.pt')
        # 최소 검증 손실 업데이트
        min_val_loss = avg_val_loss

======== Epoch 1 / 3 ========


Training Batch: 97it [00:16,  5.88it/s]



Running Validation...
  Validation Loss: 0.42
  Accuracy: 0.84
  F1 Macro: 0.80
  F1 Micro: 0.84
  F1 Weighted: 0.84
Validation loss decreased (inf --> 0.42).  Saving model ...
======== Epoch 2 / 3 ========


Training Batch: 97it [00:15,  6.37it/s]



Running Validation...
  Validation Loss: 0.39
  Accuracy: 0.83
  F1 Macro: 0.80
  F1 Micro: 0.83
  F1 Weighted: 0.83
Validation loss decreased (0.42 --> 0.39).  Saving model ...
======== Epoch 3 / 3 ========


Training Batch: 97it [00:15,  6.36it/s]



Running Validation...
  Validation Loss: 0.40
  Accuracy: 0.85
  F1 Macro: 0.81
  F1 Micro: 0.85
  F1 Weighted: 0.85


## 모델 저장과 로드

`%pwd`은 Colab에서 현재 코드가 실행되는 경로를 출력합니다.

In [ ]:
%pwd

'/content'

`%ls -al`은 Colab에서 현재 코드가 실행되는 경로에 있는 파일과 디렉토리들을 보여주는 명령어입니다.

In [ ]:
%ls -al

total 433468
drwxr-xr-x 1 root root      4096 Mar 24 02:18 ./
drwxr-xr-x 1 root root      4096 Mar 24 01:34 ../
drwxr-xr-x 4 root root      4096 Mar 21 13:23 .config/
-rw-r--r-- 1 root root   1287961 Mar 24 01:42 finance_data.csv
-rw-r--r-- 1 root root 442562069 Mar 24 02:18 model_checkpoint.pt
drwxr-xr-x 1 root root      4096 Mar 21 13:23 sample_data/


In [ ]:
# 베스트 모델 로드
model.load_state_dict(torch.load("model_checkpoint.pt"))

<All keys matched successfully>

# 테스트 데이터에 대한 평가

In [ ]:
avg_val_loss, eval_metrics = evaluate(model, test_dataloader, device)
print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))

  Test Loss: 0.36
  Accuracy: 0.85
  F1 Macro: 0.85
  F1 Micro: 0.85
  F1 Weighted: 0.85


# 예측

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512,
                return_all_scores=True, function_to_apply='softmax')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_0', 'score': 0.10445113480091095}, {'label': 'LABEL_1', 'score': 0.8862286806106567}, {'label': 'LABEL_2', 'score': 0.009320172481238842}]]


In [ ]:
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512, function_to_apply='softmax')

In [ ]:
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)

[{'label': 'LABEL_1', 'score': 0.8862286806106567}]


In [ ]:
label_dict = {'LABEL_0' : '중립', 'LABEL_1' : '긍정', 'LABEL_2' : '부정'}

In [ ]:
def prediction(text):
  result = pipe(text)

  return [label_dict[result[0]['label']]]

In [ ]:
prediction('러닝스푼즈가 매출이 급성장하였다')

['긍정']

In [ ]:
prediction('ChatGPT의 등장으로 인공지능 스타트업들은 위기다')

['부정']

In [ ]:
prediction('인공지능 기술의 발전으로 누군가는 기회를 얻을 것이고, 누군가는 얻지 못할 것이다')

['중립']